In [ ]:
from mace.calculators import mace_mp
from ase import build

from ase.md import Langevin
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase import units
from ase import io
from ase.optimize import BFGS, LBFGS, FIRE, LBFGSLineSearch
# import fire



macemp = mace_mp(dispersion=True, default_dtype="float64") # return ASE calculator
#macemp = mace_mp()
macemp_omat = mace_mp(model="/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/mace-omat-0-medium.model", dispersion=True, default_dtype="float64")

/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/e3nn/o3/_wigner.py:10: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  _Jd, _W3j_flat, _W3j_indices = torch.load(os.path.join(os.path.dirname(__file__), 'constants.pt'))


cuequivariance or cuequivariance_torch is not available. Cuequivariance acceleration will be disabled.
Using medium MPA-0 model as default MACE-MP model, to use previous (before 3.10) default model please specify 'medium' as model argument
Using Materials Project MACE for MACECalculator with /Users/joehart/.cache/mace/macempa0mediummodel
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.
Using TorchDFTD3Calculator for D3 dispersion corrections
Using float64 for MACECalculator, which is slower but more accurate. Recommended for geometry optimization.


/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)
/Users/joehart/Desktop/0_Cambridge/0_MPhil_Scientific_Computing/Written_assignments/MACE-MP-0/Notebooks_mace/mace_test/venv_mace/lib/python3.12/site-packages/mace/calculators/mace.py:139: UserWarning: Environment variable TORCH_FORCE_NO_WEIGHTS_ONLY_LOAD detected, since the`weights_only` argument was not explicitly passed to `torch.load`, forcing weights_only=False.
  torch.load(f=model_path, map_location=device)


Using TorchDFTD3Calculator for D3 dispersion corrections


In [15]:
from ase import Atoms
from ase.build import bulk
from ase.visualize import view
import numpy as np

# Define lattice parameter
a = 3.03  # Lattice constant in Å

# Define fractional coordinates for the Ti2CO2 monolayer
# Based on the described structure:
# - C atoms are sandwiched between Ti layers
# - O atoms terminate both sides

# Basis atoms in fractional coordinates
# positions = [
#     (0.0, 0.0, 0.5),  # C (middle layer)
#     (1/3, 2/3, 0.35),  # Ti (bottom layer)
#     (2/3, 1/3, 0.65),  # Ti (top layer)
#     (1/3, 2/3, 0.80),  # O (top termination)
#     (2/3, 1/3, 0.20),  # O (bottom termination)
# ]

positions = [
    (0.0, 0.0, 0.5),  # C (middle layer)
    (1/3, 2/3, 0.65),  # Ti (was top, now bottom)
    (2/3, 1/3, 0.35),  # Ti (was bottom, now top)
    (1/3, 2/3, 0.20),  # O (was top, now bottom)
    (2/3, 1/3, 0.80),  # O (was bottom, now top)
]


# Atom types corresponding to positions
symbols = ["C", "Ti", "Ti", "O", "O"]

# Define hexagonal unit cell
cell = [
    [a, 0, 0],
    [-a/2, np.sqrt(3)/2 * a, 0],
    [0, 0, 20]
]

# Create the MXene monolayer
mxene_unit = Atoms(symbols=symbols, scaled_positions=positions, cell=cell, pbc=[True, True, False])

for atom in mxene_unit:
    if atom.symbol == 'O':
        if atom.position[2] < 0.5 * cell[2][2]:  # bottom O layer, fraction x cell parameter
            atom.tag = 1
        else:  # top O layer
            atom.tag = 3
    elif atom.symbol == 'C':
        atom.tag = 2
        


from ase.build import add_vacuum

mxene_supercell = mxene_unit.repeat((4, 4, 1))
        
add_vacuum(mxene_supercell, 20.0)
# with the periodic boundary conditions in the z-direction, the vacuum layer makes sure that the interactions in the z-direction are negligible

# Visualize the structure
view(mxene_supercell, viewer='x3d')

In [11]:
mxene_supercell.calc = macemp

# Optimize the structure
optimiser = BFGS(mxene_supercell)
optimiser.run(fmax=0.0001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 22:48:37     -352.975165        1.109804
BFGS:    1 22:48:39     -353.620086        1.147950
BFGS:    2 22:48:40     -363.044624        1.703144
BFGS:    3 22:48:41     -375.411446        2.354899
BFGS:    4 22:48:43     -391.570896        3.030360
BFGS:    5 22:48:44     -411.960333        3.563458
BFGS:    6 22:48:45     -436.463937        4.097895
BFGS:    7 22:48:47     -466.481801        4.737531
BFGS:    8 22:48:48     -503.614741        5.409999
BFGS:    9 22:48:50     -547.098674        5.418499
BFGS:   10 22:48:51     -581.425634        3.287227
BFGS:   11 22:48:52     -594.005520        1.153212
BFGS:   12 22:48:53     -595.725965        1.562688
BFGS:   13 22:48:55     -602.534997        2.716427
BFGS:   14 22:48:56     -610.236496        3.813888
BFGS:   15 22:48:57     -617.449252        4.608378
BFGS:   16 22:48:58     -623.447506        5.086799
BFGS:   17 22:49:00     -629.441097        5.543421
BFGS:   18 22:

True

In [12]:
io.write("structures/mxene_relaxed_flip_mace.xyz", mxene_supercell)

In [13]:
mxene_relaxed_flip_mace = io.read("structures/mxene_relaxed_flip_mace.xyz")

In [ ]:
view(mxene_relaxed_flip_mace, viewer='x3d')

In [16]:
from ase.optimize import LBFGS, BFGS, FIRE

# use float64
mxene_supercell.calc = macemp_omat

# Optimize the structure
optimiser = BFGS(mxene_supercell)
optimiser.run(fmax=0.0001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 22:50:25     -347.785409        1.373303
BFGS:    1 22:50:26     -348.661299        1.417086
BFGS:    2 22:50:27     -359.345222        1.910915
BFGS:    3 22:50:29     -373.768063        2.480044
BFGS:    4 22:50:30     -392.902527        3.150714
BFGS:    5 22:50:31     -418.796438        4.093520
BFGS:    6 22:50:32     -455.414281        5.299871
BFGS:    7 22:50:33     -502.143006        5.661777
BFGS:    8 22:50:34     -546.701832        4.461769
BFGS:    9 22:50:35     -574.879111        2.139687
BFGS:   10 22:50:37     -580.207984        1.004369
BFGS:   11 22:50:38     -580.742411        0.626684
BFGS:   12 22:50:40     -581.086883        0.456016
BFGS:   13 22:50:41     -582.631283        0.965998
BFGS:   14 22:50:42     -586.282828        1.798029
BFGS:   15 22:50:43     -589.989227        2.597332
BFGS:   16 22:50:44     -593.910752        3.375188
BFGS:   17 22:50:46     -598.023636        4.174395
BFGS:   18 22:

True

In [17]:
io.write("structures/mxene_relaxed_flip_omat.xyz", mxene_supercell)

In [19]:
mxene_relaxed_flip_omat = io.read("structures/mxene_relaxed_flip_omat.xyz")

In [ ]:
view(mxene_relaxed_flip_omat, viewer='x3d')

### Heterostructures

In [20]:
graphene_atoms = io.read("structures/graphene_atoms.xyz")
go_relax = io.read("structures/go_omat_relax.xyz")
goh_relax = io.read("structures/goh_omat_relax.xyz")

In [21]:
def create_heterostructure(mxene, graphene_layer, height):
    # Stack MXene and graphene-based layer with a given interlayer spacing (height)
    graphene_layer = graphene_layer.copy()
    
    mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.tag == 3])

    #mxene_avg_top_C_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'C'])
    #mxene_avg_top_O_z = np.mean([atom.position[2] for atom in mxene if atom.symbol == 'O' and atom.position[2] > mxene_avg_top_C_z])
    
    graphene_layer.positions[:, 2] += mxene_avg_top_O_z - np.average(graphene_layer.positions[:, 2]) + height
    heterostructure = mxene + graphene_layer
    print("MXene avg top O z: ", mxene_avg_top_O_z)

    return heterostructure


m_g = create_heterostructure(mxene_relaxed_flip_omat, graphene_atoms, 4.5)
m_go = create_heterostructure(mxene_relaxed_flip_omat, go_relax, 4.5)
m_goh = create_heterostructure(mxene_relaxed_flip_omat, goh_relax, 4.5)


MXene avg top O z:  12.23008024
MXene avg top O z:  12.23008024
MXene avg top O z:  12.23008024


In [22]:
m_g_relaxed_flip = m_g.copy()
m_g_relaxed_flip.calc = macemp_omat
optimiser = BFGS(m_g_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 23:22:38    -1195.138717       10.045465
BFGS:    1 23:22:41    -1196.946404        8.293536
BFGS:    2 23:22:44    -1201.057984        2.613151
BFGS:    3 23:22:48    -1201.426342        1.850205
BFGS:    4 23:22:51    -1201.644150        0.566620
BFGS:    5 23:22:54    -1201.725037        0.375005
BFGS:    6 23:22:58    -1201.752523        0.233075
BFGS:    7 23:23:01    -1201.762850        0.161134
BFGS:    8 23:23:05    -1201.777415        0.194589
BFGS:    9 23:23:08    -1201.791078        0.174943
BFGS:   10 23:23:11    -1201.796550        0.108784
BFGS:   11 23:23:15    -1201.800282        0.090098
BFGS:   12 23:23:17    -1201.806314        0.175593
BFGS:   13 23:23:20    -1201.819380        0.275295
BFGS:   14 23:23:23    -1201.843586        0.404441
BFGS:   15 23:23:27    -1201.885664        0.493564
BFGS:   16 23:23:30    -1201.954960        0.525340
BFGS:   17 23:23:34    -1202.056956        0.533239
BFGS:   18 23:

True

In [23]:
io.write("structures/m_g_relaxed_flip_omat.xyz", m_g_relaxed_flip)

In [24]:
m_go_relaxed_flip = m_go.copy()
m_go_relaxed_flip.calc = macemp_omat
optimiser = BFGS(m_go_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 23:26:00    -1198.815783       10.045842
BFGS:    1 23:26:03    -1200.618660        8.446583
BFGS:    2 23:26:07    -1204.828049        2.625972
BFGS:    3 23:26:11    -1205.216361        1.896965
BFGS:    4 23:26:14    -1205.444507        0.577969
BFGS:    5 23:26:17    -1205.527476        0.392260
BFGS:    6 23:26:20    -1205.559463        0.249987
BFGS:    7 23:26:24    -1205.571199        0.168934
BFGS:    8 23:26:27    -1205.588175        0.219596
BFGS:    9 23:26:30    -1205.603982        0.190938
BFGS:   10 23:26:33    -1205.611419        0.155170
BFGS:   11 23:26:38    -1205.616446        0.134486
BFGS:   12 23:26:42    -1205.623493        0.193031
BFGS:   13 23:26:45    -1205.636079        0.262159
BFGS:   14 23:26:52    -1205.654172        0.320832
BFGS:   15 23:26:55    -1205.676436        0.308259
BFGS:   16 23:26:58    -1205.702381        0.319912
BFGS:   17 23:27:02    -1205.731675        0.352561
BFGS:   18 23:

True

In [25]:
io.write("structures/m_go_relaxed_flip_omat.xyz", m_go_relaxed_flip)

In [27]:
m_goh_relaxed_flip = m_goh.copy()
m_goh_relaxed_flip.calc = macemp_omat
optimiser = LBFGS(m_goh_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

       Step     Time          Energy          fmax
LBFGS:    0 00:15:33    -1202.682224       10.007039
LBFGS:    1 00:15:37    -1204.652443        8.182896
LBFGS:    2 00:15:40    -1208.569730        2.585768
LBFGS:    3 00:15:43    -1208.962591        1.866438
LBFGS:    4 00:15:46    -1209.186508        0.793856
LBFGS:    5 00:15:49    -1209.294043        0.551255
LBFGS:    6 00:15:52    -1209.339335        0.543623
LBFGS:    7 00:15:56    -1209.378330        0.626009
LBFGS:    8 00:15:59    -1209.463440        0.864129
LBFGS:    9 00:16:02    -1209.535515        0.806899
LBFGS:   10 00:16:05    -1209.627502        0.472252
LBFGS:   11 00:16:08    -1209.665107        0.402824
LBFGS:   12 00:16:11    -1209.708207        0.477774
LBFGS:   13 00:16:14    -1209.750994        0.396246
LBFGS:   14 00:16:17    -1209.777739        0.888159
LBFGS:   15 00:16:20    -1209.799035        0.321795
LBFGS:   16 00:16:23    -1209.813137        0.347643
LBFGS:   17 00:16:25    -1209.835361        0.51

True

In [28]:
io.write("structures/m_goh_relaxed_flip_omat.xyz", m_goh_relaxed_flip)

In [29]:
m_g_relaxed_flip_omat = io.read("structures/m_g_relaxed_flip_omat.xyz")
m_go_relaxed_flip_omat = io.read("structures/m_go_relaxed_flip_omat.xyz")
m_goh_relaxed_flip_omat = io.read("structures/m_goh_relaxed_flip_omat.xyz")

In [31]:
from ase import Atom

def add_sodium(heterostructure):
    
    heterostructure = heterostructure.copy()
    
    graphene_C_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 10]
    mxene_O_atoms = [atom.position[2] for atom in heterostructure if atom.tag == 3]
    
    z_mxene = np.mean(mxene_O_atoms)
    z_graphene = np.mean(graphene_C_atoms)

    # place Na ion halfway between MXene and graphene
    na_z = z_mxene + 0.5 * (z_graphene - z_mxene)

    # find the center of the heterostructure to place Na
    positions = heterostructure.get_positions()
    x_center = np.mean(positions[:, 0])  # x mean
    y_center = np.mean(positions[:, 1])  # y mean


    na_position = [x_center, y_center, na_z]

    # Add Na atom
    heterostructure += Atom('Na', na_position)
    # add tag
    heterostructure[-1].tag = 20

    return heterostructure

m_g_na = add_sodium(m_g_relaxed_flip_omat)
m_go_na = add_sodium(m_go_relaxed_flip_omat)
m_goh_na = add_sodium(m_goh_relaxed_flip_omat)

In [32]:
m_g_na_relaxed_flip = m_g_na.copy()
m_g_na_relaxed_flip.calc = macemp_omat
optimiser = BFGS(m_g_na_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 00:37:40    -1204.507235        4.224705
BFGS:    1 00:37:44    -1204.859296        3.119757
BFGS:    2 00:37:47    -1205.279180        1.353192
BFGS:    3 00:37:50    -1205.371337        0.861167
BFGS:    4 00:37:53    -1205.438323        0.685255
BFGS:    5 00:37:57    -1205.484114        0.675803
BFGS:    6 00:38:00    -1205.509451        0.625489
BFGS:    7 00:38:04    -1205.532422        0.481317
BFGS:    8 00:38:07    -1205.555713        0.462622
BFGS:    9 00:38:11    -1205.572839        0.461289
BFGS:   10 00:38:15    -1205.586898        0.342107
BFGS:   11 00:38:18    -1205.600831        0.283629
BFGS:   12 00:38:21    -1205.612290        0.219884
BFGS:   13 00:38:25    -1205.619469        0.112988
BFGS:   14 00:38:30    -1205.624303        0.104004
BFGS:   15 00:38:34    -1205.628526        0.115795
BFGS:   16 00:38:38    -1205.632172        0.106911
BFGS:   17 00:38:42    -1205.635106        0.076528
BFGS:   18 00:

True

In [33]:
io.write("structures/m_g_na_relaxed_flip.xyz", m_g_na_relaxed_flip)

In [34]:
m_go_na_relaxed_flip = m_go_na.copy()
m_go_na_relaxed_flip.calc = macemp_omat
optimiser = BFGS(m_go_na_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 00:42:01    -1208.695059        2.476858
BFGS:    1 00:42:04    -1208.852486        2.099011
BFGS:    2 00:42:07    -1209.088268        1.122248
BFGS:    3 00:42:10    -1209.143136        0.848755
BFGS:    4 00:42:12    -1209.230729        0.707795
BFGS:    5 00:42:16    -1209.275381        0.695402
BFGS:    6 00:42:19    -1209.300196        0.691888
BFGS:    7 00:42:22    -1209.325321        0.612977
BFGS:    8 00:42:25    -1209.356119        0.557413
BFGS:    9 00:42:28    -1209.381192        0.582494
BFGS:   10 00:42:31    -1209.402077        0.478814
BFGS:   11 00:42:34    -1209.425176        0.310053
BFGS:   12 00:42:37    -1209.443532        0.286835
BFGS:   13 00:42:40    -1209.455571        0.194266
BFGS:   14 00:42:43    -1209.464553        0.195840
BFGS:   15 00:42:46    -1209.472548        0.200384
BFGS:   16 00:42:49    -1209.480094        0.181957
BFGS:   17 00:42:52    -1209.486263        0.144137
BFGS:   18 00:

True

In [ ]:
io.write("structures/m_go_na_relaxed_flip.xyz", m_go_na_relaxed_flip)

In [36]:
m_goh_na_relaxed_flip = m_goh_na.copy()
m_goh_na_relaxed_flip.calc = macemp_omat
optimiser = BFGS(m_goh_na_relaxed_flip)
optimiser.run(fmax=0.001, steps=5000)

      Step     Time          Energy          fmax
BFGS:    0 00:46:20    -1213.666016        3.339198
BFGS:    1 00:46:24    -1213.907975        2.669853
BFGS:    2 00:46:27    -1214.260121        1.269695
BFGS:    3 00:46:30    -1214.337899        0.835072
BFGS:    4 00:46:33    -1214.409368        0.727602
BFGS:    5 00:46:36    -1214.450595        0.603672
BFGS:    6 00:46:39    -1214.473093        0.583017
BFGS:    7 00:46:42    -1214.494303        0.483635
BFGS:    8 00:46:45    -1214.516677        0.437890
BFGS:    9 00:46:48    -1214.533914        0.472991
BFGS:   10 00:46:51    -1214.548419        0.378290
BFGS:   11 00:46:54    -1214.564333        0.334116
BFGS:   12 00:46:57    -1214.578511        0.272024
BFGS:   13 00:47:00    -1214.588176        0.152576
BFGS:   14 00:47:03    -1214.594903        0.144188
BFGS:   15 00:47:06    -1214.600760        0.136978
BFGS:   16 00:47:09    -1214.605857        0.133776
BFGS:   17 00:47:12    -1214.609859        0.094493
BFGS:   18 00:

True

In [37]:
io.write("structures/m_goh_na_relaxed_flip.xyz", m_goh_na_relaxed_flip)